In [20]:
%logstop
%logstart -rtq ~/.logs/pw.py append
import seaborn as sns
sns.set()

In [3]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

--2021-12-01 08:51:27--  http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.216.8.163
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.216.8.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10367709 (9.9M) [application/json]
Saving to: ‘./pw-data/201701scripts_sample.json.gz’

     0K .......... .......... .......... .......... ..........  0% 3.56M 3s
    50K .......... .......... .......... .......... ..........  0% 7.25M 2s
   100K .......... .......... .......... .......... ..........  1% 6.85M 2s
   150K .......... .......... .......... .......... ..........  1% 46.9M 1s
   200K .......... .......... .......... .......... ..........  2% 8.61M 1s
   250K .......... .......... .......... .......... ..........  2% 64.2M 1s
   300K .......... .......... .......... .......... ..........  3% 53.9M 1s
   350K

## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [7]:
import gzip
import simplejson as json

In [8]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [9]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [10]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [11]:

import math

def describe(key):
    
    n = len(scripts)
    
    #mean
    total = sum([script[key] for script in scripts])
    avg = total/n
    
    #standard deviation
    s = math.sqrt(sum([(script[key] - avg)**2 for script in scripts])/(n-1))
    
    #median
    sorted_set = sorted([script[key] for script in scripts])
    med = (sorted_set[n//2]+sorted_set[n//2 + 1])/2 if n%2 == 0 else sorted_set[math.ceil(n/2)]
    
    #first and third quartiles
    lower_half = sorted_set[: math.floor(n/2) + 1]
    upper_half = sorted_set[math.ceil(n/2) + 1 :]
    m = len(lower_half)
    q25 = (lower_half[m//2]+lower_half[m//2 + 1])/2
    q75 = (upper_half[m//2]+upper_half[m//2 + 1])/2

    return [total, avg, s, q25, med, q75]

In [13]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [14]:
grader.score.pw__summary_statistics(summary)

Your score: 1.000


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [16]:
bnf_names = list(set([script['bnf_name'] for script in scripts]))
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [17]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    groups[script['bnf_name']].append(script)

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [18]:
bnf_items = [(name, sum([group['items'] for group in groups[name]])) for name in groups.keys()]

In [21]:
max_item = [max(bnf_items, key= lambda x : x[1])]


In [22]:
max_item


[('Omeprazole_Cap E/C 20mg', 113826)]

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [23]:
grader.score.pw__most_common_item(max_item)

Your score: 1.000


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name',)` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [26]:
def group_by_field(data, fields):
    
    groups = {}
    
    for dic in data:
        key = dic[fields[0]] if len(fields) == 1 else (dic[fields[0]], dic[fields[1]])
        if key in groups:
            groups[key].append(dic)
        else:
            groups[key] = [dic]
                
    return groups

In [27]:
groups = group_by_field(scripts, ('bnf_name',))
bnf_items = [(key, sum([group['items'] for group in groups[key]])) for key in groups.keys()]
test_max_item = [max(bnf_items, key= lambda x : x[1])]

assert test_max_item == max_item

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [28]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal :
        if practice['post_code'] < practice_postal[practice['code']]:
            practice_postal[practice['code']] = practice['post_code']
    else:
        practice_postal[practice['code']] = practice['post_code']

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [29]:
groupsByCode = group_by_field(practices, ('code',))
#post_codes = [sorted([practice['post_code'] for practice in groupsByCode[code]])[0] for code in groupsByCode.keys()]
#practice_postal = {key:value for (key,value) in zip(groupsByCode.keys(), post_codes)}
practice_postal = {key:value for key in groupsByCode.keys() \
                   for value in sorted([group['post_code'] for group in groupsByCode[key]])[:1]}

In [30]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [31]:
joined = scripts[:]
for script in joined:
    script['post_code'] = practice_postal[script['practice']]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [32]:
groups = group_by_field(joined, ('post_code',))
items_by_post = [(name, sum([group['items'] for group in groups[name]])) for name in groups.keys()]

In [33]:
items_by_post[:5]

[('SK11 6JL', 110071),
 ('CW5 5NX', 38797),
 ('CW1 3AW', 64104),
 ('CW7 1AT', 43164),
 ('CH65 6TG', 25090)]

In [34]:
postal_totals = sorted(items_by_post)[:100]


grader.score.pw__postal_totals(postal_totals)

Your score: 1.000


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

There are several approaches to solve this problem but we will guide you through one of them. Feel free to solve it your own way if it is easier for you to understand and implement. If your kernel keeps on dying, it's probably an indication that you are running out of memory. Consider deleting objects you don't need anymore using the `del` statement and shutdown any other running notebooks. For example:
```Python
del some_object_not_needed
```

The first step is to calculate the total items for each `'post_code'` and `'bnf_name'`. Let's call that result `total_items_by_post_bnf`. Consider what is the best data structure(s) to represent `total_items_by_post_bnf`. It should have 141196 `('post_code', 'bnf_name')` groups.

In [36]:
groups = group_by_field(joined, ('post_code', 'bnf_name'))
total_items_by_bnf_post = [(name, sum([group['items'] for group in groups[name]])) for name in groups.keys()]
assert len(total_items_by_bnf_post) == 141196

In [37]:
total_items_by_bnf_post[:10]


[(('SK11 6JL', 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F'), 5),
 (('SK11 6JL', 'Alginate_Raft-Forming Oral Susp S/F'), 3),
 (('SK11 6JL', 'Sod Algin/Pot Bicarb_Susp S/F'), 94),
 (('SK11 6JL', 'Sod Alginate/Pot Bicarb_Tab Chble 500mg'), 9),
 (('SK11 6JL', 'Gaviscon Infant_Sach 2g (Dual Pack) S/F'), 41),
 (('SK11 6JL', 'Gaviscon Advance_Liq (Aniseed) (Reckitt)'), 98),
 (('SK11 6JL', 'Gaviscon Advance_Tab Chble Mint(Reckitt)'), 16),
 (('SK11 6JL', 'Gaviscon Advance_Liq (Peppermint) S/F'), 65),
 (('SK11 6JL', 'Peptac_Liq (Peppermint) S/F'), 14),
 (('SK11 6JL', 'Alverine Cit_Cap 60mg'), 10)]

Next, let's take `total_items_by_post_bnf` and group it by `'post_code'`. In other words, we want a  data structure that maps a `'post_code'` to a list of all records that belong to that `'post_code'`. There should be 118 groups.

In [38]:
total_items = [{'post_code':t[0][0], 'bnf_name':t[0][1], 'items': t[1]} for t in total_items_by_bnf_post ]
total_items_by_post = group_by_field(total_items, ('post_code',))
assert len(total_items_by_post) == 118

Now with `grouped_post_code`, let's iterate over each group and calculate the following fields for each `'post_code'`:
1. the sum of total items for all `'bnf_name'`
1. the most total items
1. the `'bnf_name'` that had the most total items

Once again, consider the best data structure(s) to use to represent the result. It may help to write and use a function when developing your solution.

In [39]:
max_item_by_post = {k:value for k in total_items_by_post.keys() \
                    for value in sorted(total_items_by_post[k], key=lambda x:(x['items'], x['bnf_name']))}

Now, we are ready to:
1. calculate the ratio (the amount dispensed as proportion of total)
1. [sort](https://docs.python.org/3/howto/sorting.html) alphabetically by the post code
1. format the answer as a list of tuples
1. take only the first 100 tuples
1. submit to the grader

In [40]:
#sorted_max_item_by_post = sorted(list(max_item_by_post.items()))

items_by_region = sorted([(max_item['post_code'], max_item['bnf_name'], max_item['items']/total_items[1]) \
                   for max_item,total_items in zip(list(max_item_by_post.values()), items_by_post)])[:100]

In [41]:
grader.score.pw__items_by_region(items_by_region)

Your score: 1.000


*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*